## SETUP + reading tables

In [80]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import altair as alt
import os.path
import getpass
import platform
from pandasql import sqldf

username = getpass.getuser()

print("Username: ", username)
pathPart1 = "/Users/"
pathPart2 = "/Github/SCGProject/Datasets/"
complePath = pathPart1+username+pathPart2

if platform.system() == "Darwin":
    if(username == "marcovinciguerra"):
        dfCambio = pd.read_csv(
            complePath+"/CorrectedDatasets/dfCambio.csv")

        dfClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dfClienti.csv")

        dfConsumi = pd.read_csv(
            complePath + "/CorrectedDatasets/dfConsumi.csv")

        dfCostoOrarioConsuntivo = pd.read_csv(
            complePath+"/CorrectedDatasets/dfConsuntivo.csv")

        dfCostoOrarioBudget = pd.read_csv(
            complePath+"CorrectedDatasets/dfCostoOrario.csv")

        dfImpiegoRisorse = pd.read_csv(
            complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

        dftuttiClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dftuttiClienti.csv")

        dfVendite = pd.read_csv(
            complePath+"/CorrectedDatasets/dfVendite.csv")

    elif(username == "davidguzman"):
        pathPart1 = "/Users/"
        pathPart2 = "/documents/Github/SCGProject/Datasets/"
        complePath = pathPart1+username+pathPart2

        dfCambio = pd.read_csv(
            complePath+"/CorrectedDatasets/dfCambio.csv")

        dfClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dfClienti.csv")

        dfConsumi = pd.read_csv(
            complePath + "/CorrectedDatasets/dfConsumi.csv")

        dfCostoOrarioConsuntivo = pd.read_csv(
            complePath+"/CorrectedDatasets/dfConsuntivo.csv")

        dfCostoOrarioBudget = pd.read_csv(
            complePath+"CorrectedDatasets/dfCostoOrario.csv")

        dfImpiegoRisorse = pd.read_csv(
            complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

        dftuttiClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dftuttiClienti.csv")

        dfVendite = pd.read_csv(
            complePath+"/CorrectedDatasets/dfVendite.csv")

elif platform.system() == "Linux":
    complePath = "/home/alinux/unibg/git_repo/SCGProject"
    pathPart1 = "/Users/"
    pathPart2 = "/documents/Github/SCGProject/Datasets/"
    complePath = pathPart1+username+pathPart2

    dfCambio = pd.read_csv(
        complePath+"/CorrectedDatasets/dfCambio.csv")

    dfClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dfClienti.csv")

    dfConsumi = pd.read_csv(
        complePath + "/CorrectedDatasets/dfConsumi.csv")

    dfCostoOrarioConsuntivo = pd.read_csv(
        complePath+"/CorrectedDatasets/dfConsuntivo.csv")

    dfCostoOrarioBudget = pd.read_csv(
        complePath+"CorrectedDatasets/dfCostoOrario.csv")

    dfImpiegoRisorse = pd.read_csv(
        complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

    dftuttiClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dftuttiClienti.csv")

    dfVendite = pd.read_csv(
        complePath+"/CorrectedDatasets/dfVendite.csv")


Username:  marcovinciguerra


# Totale dei ricavi considerando anche la valuta locale


In [81]:
# Vendite
dfVendite.head()

,Unnamed: 0,NrMovimento,budget/cons,NrArticolo,NrOrigine,Quantity,ImportoVenditaValutaLocaleTOTALEVENDITA
0,0,21621,BUDGET,ART0001433,C00140,4,18.80
1,1,21629,BUDGET,ART0001610,C00140,12,145.28
2,2,21631,BUDGET,ART0003269,C00140,24,933.99
3,3,21633,BUDGET,ART0002249,C00140,1,36.33
4,4,21634,BUDGET,ART0002370,C00140,1,65.72


In [82]:
# Valuta locale 
dfCambio = dfCambio.stack().str.replace(',', '.').unstack()


dfCambio.head()


,Unnamed: 0,CodiceValuta,Anno,TassoCambioMedio
0,NaN,NaN,BUDGET,1
1,NaN,NaN,BUDGET,1.0541
2,NaN,NaN,BUDGET,123.4
3,NaN,NaN,CONSUNTIVO,1
4,NaN,NaN,CONSUNTIVO,1.1993


# Costruzione della tabella con i cammini di join

In [83]:
dfVendite.rename(columns={'budget/cons': 'budget'}, inplace=True)
dfVendite.loc[dfVendite.loc[:, "budget"] ==
              "Consuntivo", "budget"] = "CONSUNTIVO"
dfVendite.loc[dfVendite.loc[:, "budget"]
              == "Budget", "budget"] = "BUDGET"


#Joining with the exchange rate
VenditeTotale = sqldf(
    "SELECT * FROM  dfClienti C JOIN dfVendite V ON C.Nr = V.Nr") #DEVO SISTEMARE LA COLONNA

#La prima colonna è il codice del cliente
VenditeTotale = sqldf(
    "SELECT * FROM VenditeTotale V JOIN dfCambio C ON  V.Valuta = C.CodiceValuta AND V.budget = C.Anno ")
del VenditeTotale['Anno']
del VenditeTotale['CodiceValuta']

VenditeTotale


PandaSQLException: (sqlite3.OperationalError) no such column: V.Nr
[SQL: SELECT * FROM  dfClienti C JOIN dfVendite V ON C.Nr = V.Nr]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

## Somma per il budget

In [ ]:
totVenditeBudget = sqldf(
    "SELECT SUM(ImportoVenditaValutaLocaleTOTALEVENDITA/TassoCambioMedio) FROM VenditeTotale WHERE budget = 'BUDGET' ")
print(totVenditeBudget)


   SUM(ImportoVenditaValutaLocaleTOTALEVENDITA/TassoCambioMedio)
0                                      414810.859303            


## Somma per il consuntivo

In [ ]:
totVenditeConsuntivo = sqldf(
    "SELECT SUM(ImportoVenditaValutaLocaleTOTALEVENDITA/TassoCambioMedio) FROM VenditeTotale WHERE budget = 'CONSUNTIVO' ")
print(totVenditeConsuntivo)


   SUM(ImportoVenditaValutaLocaleTOTALEVENDITA/TassoCambioMedio)
0                                      596811.749407            


## Calcolo dello scostamento

In [ ]:
ScostamentoVendite = totVenditeConsuntivo - totVenditeBudget
print(ScostamentoVendite)


   SUM(ImportoVenditaValutaLocaleTOTALEVENDITA/TassoCambioMedio)
0                                      182000.890104            


# Totale dei costi

In [ ]:
dfConsumi.head()

,Unnamed: 0,NrMovimento,Budget/cons,CodiceMP,NrArticolo,NrDocumento,QuantityMPImpiegata,ImportoCostoTOTALE
0,0,28854,BUDGET,ART0001074,ART0000018,ODP17-05342,30.0,5.15
1,1,66819,CONSUNTIVO,ART0001074,ART0000018,ODP19-05998,30.0,5.15
2,2,28853,BUDGET,ART0005696,ART0000041,ODP17-05342,4.0,92.00
3,3,66818,CONSUNTIVO,ART0005696,ART0000041,ODP19-05998,4.0,88.00
4,4,28855,BUDGET,ART0005697,ART0000042,ODP17-05342,4.0,104.00


### Impiego orario delle risorse

In [ ]:
dfImpiegoRisorse.head()

,Unnamed: 0,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,QuantitydiOutput
0,0,ART0000018,BUDGET,ODP17-05342,Tornitura,A20,RIS2,1.00,2.0
1,1,ART0000018,BUDGET,ODP17-05342,Fresatura,A30,RIS7,0.75,2.0
2,2,ART0000018,BUDGET,ODP17-05342,Controllo qualità,CQ,RIS16,0.00,2.0
3,3,ART0000018,CONSUNTIVO,ODP19-05998,Tornitura,A20,RIS2,1.00,2.0
4,4,ART0000018,CONSUNTIVO,ODP19-05998,Fresatura,A30,RIS5,1.00,2.0


### Costo orario delle risorse a consuntivo

In [ ]:
dfCostoOrarioConsuntivo.head()

,Unnamed: 0,Risorsa,AreaProduzione,CostoOrario
0,0,RIS1,A10,60
1,1,RIS1,A11,60
2,2,RIS1,A20,80
3,3,RIS1,A30,80
4,4,RIS1,A32,60


### Costo orario delle risorse a budget

In [ ]:
dfCostoOrarioConsuntivo.head()

,Unnamed: 0,Risorsa,AreaProduzione,CostoOrario
0,0,RIS1,A10,60
1,1,RIS1,A11,60
2,2,RIS1,A20,80
3,3,RIS1,A30,80
4,4,RIS1,A32,60


In [ ]:
dfImpiegoRisorse.head()


,Unnamed: 0,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,QuantitydiOutput
0,0,ART0000018,BUDGET,ODP17-05342,Tornitura,A20,RIS2,1.00,2.0
1,1,ART0000018,BUDGET,ODP17-05342,Fresatura,A30,RIS7,0.75,2.0
2,2,ART0000018,BUDGET,ODP17-05342,Controllo qualità,CQ,RIS16,0.00,2.0
3,3,ART0000018,CONSUNTIVO,ODP19-05998,Tornitura,A20,RIS2,1.00,2.0
4,4,ART0000018,CONSUNTIVO,ODP19-05998,Fresatura,A30,RIS5,1.00,2.0


### Calcolo dei costi a budget

In [ ]:
ImportoCostoTotaleMPBudget = sqldf(
    "SELECT * FROM dfCostoOrarioBudget B JOIN dfImpiegoRisorse R ON B.Risorsa = R.Risorsa WHERE budgetConsuntivo='BUDGET' ")

ImportoCostoTotaleMPBudget.head()


,Unnamed: 0,Risorsa,AreaProduzione,CostoOrario,Unnamed: 0,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,QuantitydiOutput
0,42,RIS2,A10,51.0,0,ART0000018,BUDGET,ODP17-05342,Tornitura,A20,RIS2,1.0,2.0
1,43,RIS2,A11,51.0,0,ART0000018,BUDGET,ODP17-05342,Tornitura,A20,RIS2,1.0,2.0
2,44,RIS2,A20,68.0,0,ART0000018,BUDGET,ODP17-05342,Tornitura,A20,RIS2,1.0,2.0
3,45,RIS2,A30,51.0,0,ART0000018,BUDGET,ODP17-05342,Tornitura,A20,RIS2,1.0,2.0
4,46,RIS2,A33,42.5,0,ART0000018,BUDGET,ODP17-05342,Tornitura,A20,RIS2,1.0,2.0


### Calcolo dei costi per Budget

In [ ]:
TotaleCostoBudget = sqldf(
    "SELECT SUM(TempoRisorsa*CostoOrario) FROM ImportoCostoTotaleMPBudget WHERE budgetConsuntivo = 'BUDGET'")
print(TotaleCostoBudget)


   SUM(TempoRisorsa*CostoOrario)
0                   1.050763e+06


In [ ]:
ImportoCostoTotaleConsuntivo = sqldf(
    "SELECT * FROM dfCostoOrarioConsuntivo B JOIN dfImpiegoRisorse R ON B.Risorsa = R.Risorsa WHERE budgetConsuntivo='CONSUNTIVO'")

ImportoCostoTotaleConsuntivo.head()


,Unnamed: 0,Risorsa,AreaProduzione,CostoOrario,Unnamed: 0,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,QuantitydiOutput
0,42,RIS2,A10,60,3,ART0000018,CONSUNTIVO,ODP19-05998,Tornitura,A20,RIS2,1.0,2.0
1,43,RIS2,A11,60,3,ART0000018,CONSUNTIVO,ODP19-05998,Tornitura,A20,RIS2,1.0,2.0
2,44,RIS2,A20,80,3,ART0000018,CONSUNTIVO,ODP19-05998,Tornitura,A20,RIS2,1.0,2.0
3,45,RIS2,A30,60,3,ART0000018,CONSUNTIVO,ODP19-05998,Tornitura,A20,RIS2,1.0,2.0
4,46,RIS2,A33,50,3,ART0000018,CONSUNTIVO,ODP19-05998,Tornitura,A20,RIS2,1.0,2.0


In [ ]:
ImportoCostoTotaleMPBudget = sqldf(
    "SELECT * FROM dfCostoOrarioBudget B JOIN dfImpiegoRisorse R ON B.Risorsa = R.Risorsa WHERE budgetConsuntivo = 'CONSUNTIVO'")

ImportoCostoTotaleMPBudget.head()


,Unnamed: 0,Risorsa,AreaProduzione,CostoOrario,Unnamed: 0,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,QuantitydiOutput
0,42,RIS2,A10,51.0,3,ART0000018,CONSUNTIVO,ODP19-05998,Tornitura,A20,RIS2,1.0,2.0
1,43,RIS2,A11,51.0,3,ART0000018,CONSUNTIVO,ODP19-05998,Tornitura,A20,RIS2,1.0,2.0
2,44,RIS2,A20,68.0,3,ART0000018,CONSUNTIVO,ODP19-05998,Tornitura,A20,RIS2,1.0,2.0
3,45,RIS2,A30,51.0,3,ART0000018,CONSUNTIVO,ODP19-05998,Tornitura,A20,RIS2,1.0,2.0
4,46,RIS2,A33,42.5,3,ART0000018,CONSUNTIVO,ODP19-05998,Tornitura,A20,RIS2,1.0,2.0


In [ ]:
TotaleCostoConsuntivo = sqldf(
    "SELECT SUM(TempoRisorsa*CostoOrario) FROM ImportoCostoTotaleMPBudget WHERE budgetConsuntivo = 'CONSUNTIVO'")
print(TotaleCostoConsuntivo)


   SUM(TempoRisorsa*CostoOrario)
0                   1.626986e+06


### Calcolo della differenza

In [ ]:
scostamentoCosti = TotaleCostoConsuntivo - TotaleCostoBudget


,SUM(TempoRisorsa*CostoOrario)
0,-1.050763e+06
